# Propeller
* Run the scene workshop1.ttt in CoppeliaSim

In [238]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Step by step for the homework:
Design a vehicle in 3D. Define the position and the orientation of the motors
Compute the A matrix and check that it is full rank
Design the vehicle in CoppeliaSim by placing the propellers as children of the cube (or any other shape)
Design the position and attitude controller of the vehicle.

### Check if python is connecting to Coppelia

In [239]:
sim.simxFinish(-1)  # Close opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    print('Simulation time in milliseconds: ', sim.simxGetLastCmdTime(clientID))
    
    # Get Object position
    name = 'propeller'
    err_code, cuboid = sim.simxGetObjectHandle(clientID, name, sim.simx_opmode_blocking)
    res, position = sim.simxGetObjectPosition(clientID, cuboid, -1, sim.simx_opmode_blocking)        
#     print(name, 'is at [x,y,z]=', position)
    
    
    returnCode = sim.simxSetFloatSignal(clientID,'f', 100.0,sim.simx_opmode_oneshot)
#     returnCode, signalValue=sim.simxGetFloatSignal(clientID,'particleVelocity',sim.simx_opmode_oneshot_wait)
#     print(signalValue)
    
    # Now close the connection to CoppeliaSim:
    sim.simxGetPingTime(clientID)
    sim.simxFinish(clientID)
    print('Disconnected')
else:
    print('Failed connecting to remote API server')


Connected
Simulation time in milliseconds:  114651
Disconnected


# Robot

In [240]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_oneshot)        
        return array(position)
    
    def get_velocity(self, relative_object=-1):
        # Get velocity relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, velocity, omega = sim.simxGetObjectVelocity(self.client_id, self.frame, sim.simx_opmode_oneshot)        
        return array(velocity), array(omega)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_oneshot)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_oneshot)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_oneshot)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_oneshot)
        return array(position)
    
    def set_force(self, f, signal='f'):
        return sim.simxSetFloatSignal(clientID, signal, f, sim.simx_opmode_oneshot)
    
    def set_forces(self, f, signal='f'):
        # For many propellers
        for i, fi in enumerate(f):
            signali = signal + str(i+1)
            sim.simxSetFloatSignal(clientID, signali, fi, sim.simx_opmode_oneshot)
    
       # Added this, Was missing get orientation
    def get_orientation(self, relative_object=-1): 
        # Retrieves the orientation (Euler angles) of an object.
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, orientation = sim.simxGetObjectOrientation(self.client_id, self.frame, relative_object, sim.simx_opmode_oneshot)        
        return array(orientation)
    
        

In [241]:
# Send force to a robot
r = robot('Cuboid')  # Create an instance of our robot
r.set_force(100)

r.close_connection()  

Robot connected
Connection closed


# P-Control

In [242]:

# Rotation matrices
RotX = lambda ϕ: [[1, 0, 0], 
         [0, cos(ϕ), -sin(ϕ)],
         [0, sin(ϕ), cos(ϕ)]]
RotY = lambda θ: [[cos(θ), 0, sin(θ)], 
         [0, 1, 0],
         [-sin(θ), 0, cos(θ)]]
RotZ = lambda ψ: [[cos(ψ), -sin(ψ), 0], 
        [sin(ψ), cos(ψ), 0],
        [0, 0, 1]]
RotZYX = lambda Θ: np.dot(RotZ(Θ[2]), np.dot(RotY(Θ[1]), RotX(Θ[0])))


# Rotation Euler Angles Tait-Bryan convention - https://coppeliarobotics.com/helpFiles/en/eulerAngles.htm
import numpy as np

# Rotation around Z-axis 
def yaw_rotateX(w):
    yaw = np.array([[1, 0, 0], [0 , np.cos(w),  - np.sin(w)], [0, np.sin(w), np.cos(w)]])  
    return yaw


# Rotation around Y-axis 
def pitch_rotateY(v):
    pitch = np.array([[np.cos(v), 0 , np.sin(v)], [0, 1, 0], [-np.sin(v), 0, np.cos(v)]])   
    return pitch


# Rotation around z-axis 
def roll_rotateZ(u):
    roll = np.array([ [np.cos(u), - np.sin(u), 0], [np.sin(u), np.cos(u), 0], [0, 0, 1]])  
    return roll


In [243]:
# Propeller orientations
angles = [(0, pi/4), (0, -pi/4), (0, -pi/4), (0, pi/4), 
         (3*pi/4, 0), (3*pi/4, 0), (-3*pi/4, 0), (-3*pi/4, 0)]
R_prop = [RotZYX((ϕ, θ, 0)) for ϕ, θ in angles]
e3 = array([[0],[0],[1]])
z_prop = [np.dot(Ri, e3) for Ri in R_prop]
# Propeller locations
p = [(1, -1, 1), (-1, -1, 1), (-1, 1, 1), (1, 1, 1),
    (1, -1, -1), (-1, -1, -1), (-1, 1, -1), (1, 1, -1)]
# scale locations
d = 0.1  # distance of the arm in an axis
p = d * array(p)
# A matrix
A_force = np.hstack(z_prop)
A_torque = np.hstack([np.cross(pi, zi.T).T for pi, zi in zip(p, z_prop)])
A = np.vstack((A_force, A_torque))
print(A)
Apinv = np.linalg.pinv(A)

[[ 7.07106781e-01 -7.07106781e-01 -7.07106781e-01  7.07106781e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -7.07106781e-01 -7.07106781e-01  7.07106781e-01  7.07106781e-01]
 [ 7.07106781e-01  7.07106781e-01  7.07106781e-01  7.07106781e-01
  -7.07106781e-01 -7.07106781e-01 -7.07106781e-01 -7.07106781e-01]
 [-7.07106781e-02 -7.07106781e-02  7.07106781e-02  7.07106781e-02
  -1.38777878e-17 -1.38777878e-17  1.38777878e-17  1.38777878e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   7.07106781e-02 -7.07106781e-02 -7.07106781e-02  7.07106781e-02]
 [ 7.07106781e-02 -7.07106781e-02  7.07106781e-02 -7.07106781e-02
  -7.07106781e-02  7.07106781e-02 -7.07106781e-02  7.07106781e-02]]


In [244]:

m = 8 + 0.1 #kg mass of the block plus the mass of the prop
g = 9.8
I = 80

log = []

import math
# Send force to a robot
r = robot('Cuboid')  # Create an instance of our robot

# propellers .1 each
m = 8.8 #kg mass of the block plus the mass of the prop
g = 9.81
I = 80


log = []
print("r position:", r.get_position())
# print("d position:", d.get_position())

print("r orientation:", r.get_orientation())
wrench = [0, 0, 0, 0, 0, 0]
r.set_forces(wrench)

try:

    while True:
        # Robot state
        p = r.get_position()
        v, ω = r.get_velocity()
        theta = r.get_orientation()

        # Desired state
        p_d = [0, 0, .5] # Because we want it to be flying, in the air 
        v_d, ω_d = 0,0
        # theta_d = [0, 0, 0]
        theta_d = [0, 0, np.pi/6]


        ep = p_d - p
        ev = v_d - v
        eω = ω_d - ω
        # error in orientation
        et = theta_d - theta 
        print("ep: {}, et: {}".format(ep, et))
    


        kp, kd = .1, 1 
        a = kp * ep + kd * ev
        

        # angular acceleration
        kp_th , kd_th = 0.0004, 0.002 
        ath = kp_th * et + kd_th * eω 
        torq = I * ath 


        f = m * a
        f[2] += m*g

       # To world frame
       # https://help.naturalpoint.com/kb/articles/transform-world-space-coordinates-to-local-rigid-body-coordinates
       # R =  yaw_rotateX( theta[0])  * pitch_rotateY(theta[1]) * roll_rotateZ( theta[2] )

        R = RotZYX(theta)
        f_d = np.dot(R, f) 
        # print("u:", u)
        
        wrench = [f_d[0], f_d[1], f_d[2], torq[0], torq[1], torq[2]]
        u = np.dot(Apinv, wrench)
        r.set_forces(u.tolist())
        
        # r.wrench( [f_d[0], f_d[1], f_d[2], torq[0], torq[1], torq[2]])
        
        log.append(ep)
        time.sleep(0.001)
except KeyboardInterrupt:
    r.close_connection()
    

r.close_connection()  

Robot connected
r position: [0. 0. 0.]
r orientation: [0. 0. 0.]
ep: [0.  0.  0.5], et: [0.         0.         0.52359878]
ep: [0.  0.  0.5], et: [0.         0.         0.52359878]
ep: [0.  0.  0.5], et: [0.         0.         0.52359878]
ep: [0.  0.  0.5], et: [0.         0.         0.52359878]
ep: [0.  0.  0.5], et: [0.         0.         0.52359878]
ep: [0.  0.  0.5], et: [0.         0.         0.52359878]
ep: [ 0.10152818 -0.01243449  0.32856475], et: [0.00137907 0.00040588 0.12048613]
ep: [ 0.10152818 -0.01243449  0.32856475], et: [0.00137907 0.00040588 0.12048613]
ep: [ 0.10155635 -0.01227645  0.32808338], et: [0.00267616 0.00126235 0.12079816]
ep: [ 0.10155635 -0.01227645  0.32808338], et: [0.00267616 0.00126235 0.12079816]
ep: [ 0.10155635 -0.01227645  0.32808338], et: [0.00267616 0.00126235 0.12079816]
ep: [ 0.10160506 -0.01216763  0.32746825], et: [0.00407915 0.00210743 0.12053715]
ep: [ 0.10160506 -0.01216763  0.32746825], et: [0.00407915 0.00210743 0.12053715]
ep: [ 0.10160

ep: [ 0.29684749 -0.56118351  0.2772228 ], et: [0.02565889 0.00426784 0.07861374]
ep: [ 0.29868394 -0.58864439  0.27557679], et: [0.02506341 0.00407539 0.07773669]
ep: [ 0.29868394 -0.58864439  0.27557679], et: [0.02506341 0.00407539 0.07773669]
ep: [ 0.29868394 -0.58864439  0.27557679], et: [0.02506341 0.00407539 0.07773669]
ep: [ 0.29868394 -0.58864439  0.27557679], et: [0.02506341 0.00407539 0.07773669]
ep: [ 0.29868394 -0.58864439  0.27557679], et: [0.02506341 0.00407539 0.07773669]
ep: [ 0.29990801 -0.61630756  0.27390309], et: [0.02444876 0.00388275 0.07686846]
ep: [ 0.29990801 -0.61630756  0.27390309], et: [0.02444876 0.00388275 0.07686846]
ep: [ 0.30049375 -0.64409679  0.27220301], et: [0.02381854 0.00369099 0.07600953]
ep: [ 0.30049375 -0.64409679  0.27220301], et: [0.02381854 0.00369099 0.07600953]
ep: [ 0.30049375 -0.64409679  0.27220301], et: [0.02381854 0.00369099 0.07600953]
ep: [ 0.30049375 -0.64409679  0.27220301], et: [0.02381854 0.00369099 0.07600953]
ep: [ 0.30049375

ep: [-0.28343707 -1.33528602  0.18997353], et: [-0.00040613 -0.00138895  0.0449367 ]
ep: [-0.28343707 -1.33528602  0.18997353], et: [-0.00040613 -0.00138895  0.0449367 ]
ep: [-0.29865646 -1.32880747  0.18866864], et: [-0.00056824 -0.00139622  0.04445644]
ep: [-0.29865646 -1.32880747  0.18866864], et: [-0.00056824 -0.00139622  0.04445644]
ep: [-0.31351009 -1.32158923  0.18738168], et: [-0.00071741 -0.0014011   0.04398109]
ep: [-0.31351009 -1.32158923  0.18738168], et: [-0.00071741 -0.0014011   0.04398109]
ep: [-0.31351009 -1.32158923  0.18738168], et: [-0.00071741 -0.0014011   0.04398109]
ep: [-0.31351009 -1.32158923  0.18738168], et: [-0.00071741 -0.0014011   0.04398109]
ep: [-0.32797709 -1.31365514  0.18611217], et: [-0.00085416 -0.00140352  0.04351027]
ep: [-0.32797709 -1.31365514  0.18611217], et: [-0.00085416 -0.00140352  0.04351027]
ep: [-0.342035   -1.30503631  0.18485948], et: [-0.00097891 -0.00140375  0.0430439 ]
ep: [-0.342035   -1.30503631  0.18485948], et: [-0.00097891 -0.00

ep: [-0.44931182 -0.68137819  0.13980201], et: [-0.00017905 -0.00055339  0.02627598]
ep: [-0.44225651 -0.67117345  0.13898078], et: [-0.00013063 -0.00054008  0.02598842]
ep: [-0.44225651 -0.67117345  0.13898078], et: [-0.00013063 -0.00054008  0.02598842]
ep: [-0.43503085 -0.66136158  0.13816601], et: [-8.38324777e-05 -5.27450815e-04  2.57041011e-02]
ep: [-0.43503085 -0.66136158  0.13816601], et: [-8.38324777e-05 -5.27450815e-04  2.57041011e-02]
ep: [-0.42764503 -0.65194476  0.13735765], et: [-3.87583023e-05 -5.15578547e-04  2.54228565e-02]
ep: [-0.42764503 -0.65194476  0.13735765], et: [-3.87583023e-05 -5.15578547e-04  2.54228565e-02]
ep: [-0.42764503 -0.65194476  0.13735765], et: [-3.87583023e-05 -5.15578547e-04  2.54228565e-02]
ep: [-0.42764503 -0.65194476  0.13735765], et: [-3.87583023e-05 -5.15578547e-04  2.54228565e-02]
ep: [-0.42764503 -0.65194476  0.13735765], et: [-3.87583023e-05 -5.15578547e-04  2.54228565e-02]
ep: [-0.42764503 -0.65194476  0.13735765], et: [ 4.58405020e-06 -5

ep: [-0.16212025 -0.54826182  0.10516456], et: [ 0.00035167 -0.00046926  0.01516859]
ep: [-0.16098444 -0.54922938  0.10457692], et: [ 0.00034034 -0.00047213  0.01500224]
ep: [-0.16098444 -0.54922938  0.10457692], et: [ 0.00034034 -0.00047213  0.01500224]
ep: [-0.16098444 -0.54922938  0.10457692], et: [ 0.00034034 -0.00047213  0.01500224]
ep: [-0.16003993 -0.55015492  0.10399258], et: [ 0.00032895 -0.00047496  0.0148382 ]
ep: [-0.16003993 -0.55015492  0.10399258], et: [ 0.00032895 -0.00047496  0.0148382 ]
ep: [-0.16003993 -0.55015492  0.10399258], et: [ 0.00032895 -0.00047496  0.0148382 ]
ep: [-0.16003993 -0.55015492  0.10399258], et: [ 0.00032895 -0.00047496  0.0148382 ]
ep: [-0.16003993 -0.55015492  0.10399258], et: [ 0.00031758 -0.00047766  0.01467578]
ep: [-0.15927789 -0.55102617  0.10341156], et: [ 0.00031758 -0.00047766  0.01467578]
ep: [-0.15927789 -0.55102617  0.10341156], et: [ 0.00031758 -0.00047766  0.01467578]
ep: [-0.15868978 -0.55183226  0.10283375], et: [ 0.00030625 -0.00

...

ep: [-6.23999611e-02  1.39298588e-02 -1.23381615e-05], et: [ 1.07887536e-04 -3.64438543e-04  3.11586588e-05]
ep: [-6.23999611e-02  1.39298588e-02 -1.23381615e-05], et: [ 1.07887536e-04 -3.64438543e-04  3.11586588e-05]
ep: [-6.24016896e-02  1.39351087e-02 -1.23381615e-05], et: [ 1.07839609e-04 -3.64391221e-04  3.11586588e-05]
ep: [-6.24016896e-02  1.39351087e-02 -1.23381615e-05], et: [ 1.07839609e-04 -3.64391221e-04  3.11586588e-05]
ep: [-6.24016896e-02  1.39351087e-02 -1.23381615e-05], et: [ 1.07839609e-04 -3.64391221e-04  3.11586588e-05]
ep: [-6.24016896e-02  1.39351087e-02 -1.23381615e-05], et: [ 1.07839609e-04 -3.64391221e-04  3.11586588e-05]
ep: [-6.24034405e-02  1.39403539e-02 -1.23381615e-05], et: [ 1.07770800e-04 -3.64366657e-04  3.11586588e-05]
ep: [-6.24034405e-02  1.39403539e-02 -1.23381615e-05], et: [ 1.07770800e-04 -3.64366657e-04  3.11586588e-05]
ep: [-6.24034405e-02  1.39403539e-02 -1.23381615e-05], et: [ 1.07770800e-04 -3.64366657e-04  3.11586588e-05]
ep: [-6.24052286e-0

ep: [-6.25039488e-02  1.41987670e-02 -1.50799751e-05], et: [ 1.07596286e-04 -3.65476066e-04  3.11586588e-05]
ep: [-6.25039488e-02  1.41987670e-02 -1.50799751e-05], et: [ 1.07596286e-04 -3.65476066e-04  3.11586588e-05]
ep: [-6.25039488e-02  1.41987670e-02 -1.50799751e-05], et: [ 1.07596286e-04 -3.65476066e-04  3.11586588e-05]
ep: [-6.25060946e-02  1.42038185e-02 -1.50799751e-05], et: [ 1.07611973e-04 -3.65553133e-04  3.11586588e-05]
ep: [-6.25060946e-02  1.42038185e-02 -1.50799751e-05], et: [ 1.07611973e-04 -3.65553133e-04  3.11586588e-05]
ep: [-6.25082403e-02  1.42088700e-02 -1.50799751e-05], et: [ 1.07630352e-04 -3.65651271e-04  3.11586588e-05]
ep: [-6.25082403e-02  1.42088700e-02 -1.50799751e-05], et: [ 1.07630352e-04 -3.65651271e-04  3.11586588e-05]
ep: [-6.25082403e-02  1.42088700e-02 -1.50799751e-05], et: [ 1.07630352e-04 -3.65651271e-04  3.11586588e-05]
ep: [-6.25082403e-02  1.42088700e-02 -1.50799751e-05], et: [ 1.07630352e-04 -3.65651271e-04  3.11586588e-05]
ep: [-6.25082403e-0

ep: [-6.26443177e-02  1.44095812e-02 -1.65700912e-05], et: [ 1.09077395e-04 -3.64941981e-04  3.11586588e-05]
ep: [-6.26443177e-02  1.44095812e-02 -1.65700912e-05], et: [ 1.09077395e-04 -3.64941981e-04  3.11586588e-05]
ep: [-6.26443177e-02  1.44095812e-02 -1.65700912e-05], et: [ 1.09077395e-04 -3.64941981e-04  3.11586588e-05]
ep: [-6.26478642e-02  1.44134108e-02 -1.65700912e-05], et: [ 1.09189132e-04 -3.64831503e-04  3.11586588e-05]
ep: [-6.26478642e-02  1.44134108e-02 -1.65700912e-05], et: [ 1.09189132e-04 -3.64831503e-04  3.11586588e-05]
ep: [-6.26478642e-02  1.44134108e-02 -1.65700912e-05], et: [ 1.09189132e-04 -3.64831503e-04  3.11586588e-05]
ep: [-6.26478642e-02  1.44134108e-02 -1.65700912e-05], et: [ 1.09189132e-04 -3.64831503e-04  3.11586588e-05]
ep: [-6.26513138e-02  1.44173000e-02 -1.65700912e-05], et: [ 1.09245673e-04 -3.64740234e-04  3.11586588e-05]
ep: [-6.26513138e-02  1.44173000e-02 -1.65700912e-05], et: [ 1.09245673e-04 -3.64740234e-04  3.11586588e-05]
ep: [-6.26546592e-0

ep: [-6.27430603e-02  1.45846847e-02 -1.75237656e-05], et: [ 1.11434165e-04 -3.75630421e-04  3.18739145e-05]
ep: [-6.27472252e-02  1.45855844e-02 -1.77025795e-05], et: [ 1.11649249e-04 -3.76229145e-04  3.18739145e-05]
ep: [-6.27472252e-02  1.45855844e-02 -1.77025795e-05], et: [ 1.11649249e-04 -3.76229145e-04  3.18739145e-05]
ep: [-6.27472252e-02  1.45855844e-02 -1.77025795e-05], et: [ 1.11649249e-04 -3.76229145e-04  3.18739145e-05]
ep: [-6.27472252e-02  1.45855844e-02 -1.77025795e-05], et: [ 1.11649249e-04 -3.76229145e-04  3.18739145e-05]
ep: [-6.27516955e-02  1.45863015e-02 -1.78217888e-05], et: [ 1.11826499e-04 -3.76871903e-04  3.18739145e-05]
ep: [-6.27516955e-02  1.45863015e-02 -1.78217888e-05], et: [ 1.11955414e-04 -3.77479708e-04  3.18739145e-05]
ep: [-6.27565011e-02  1.45868333e-02 -1.78217888e-05], et: [ 1.11955414e-04 -3.77479708e-04  3.18739145e-05]
ep: [-6.27565011e-02  1.45868333e-02 -1.78217888e-05], et: [ 1.11955414e-04 -3.77479708e-04  3.18739145e-05]
ep: [-6.27616495e-0

ep: [-6.34565130e-02  1.47898272e-02 -2.28285789e-05], et: [ 1.06610110e-04 -3.81833263e-04  3.28275888e-05]
ep: [-6.34732991e-02  1.48044946e-02 -2.30669975e-05], et: [ 1.06808213e-04 -3.81870836e-04  3.28275888e-05]
ep: [-6.34732991e-02  1.48044946e-02 -2.30669975e-05], et: [ 1.06808213e-04 -3.81870836e-04  3.28275888e-05]
ep: [-6.34732991e-02  1.48044946e-02 -2.30669975e-05], et: [ 1.06808213e-04 -3.81870836e-04  3.28275888e-05]
ep: [-6.34897053e-02  1.48194255e-02 -2.33650208e-05], et: [ 1.06967935e-04 -3.81891907e-04  3.28275888e-05]
ep: [-6.34897053e-02  1.48194255e-02 -2.33650208e-05], et: [ 1.07094864e-04 -3.81924590e-04  3.28275888e-05]
ep: [-6.35057241e-02  1.48346033e-02 -2.34842300e-05], et: [ 1.07094864e-04 -3.81924590e-04  3.28275888e-05]
ep: [-6.35057241e-02  1.48346033e-02 -2.34842300e-05], et: [ 1.07094864e-04 -3.81924590e-04  3.28275888e-05]
ep: [-6.35213181e-02  1.48499962e-02 -2.35438347e-05], et: [ 1.07228690e-04 -3.81974300e-04  3.28275888e-05]
ep: [-6.35213181e-0

In [ ]:
plot(log)